<a href="https://colab.research.google.com/github/nitish2022/Product360---Market-Sentiment-Analysis/blob/master/Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Market Sentiment Analysis for the product - 'Nokia 6.1'

In [ ]:

import numpy as np
import pandas as pd

In [ ]:
import nltk
#import stop words , stemming , lematization
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [ ]:
#df1 = pd.read_csv("/home/nitish/Desktop/clean_data.csv")
from google.colab import files
uploaded = files.upload()

Saving clean_data.csv to clean_data (1).csv


In [ ]:
import io
df1 = pd.read_csv(io.StringIO(uploaded['clean_data.csv'].decode('utf-8')) , sep = ',')
df1.head()

,sentiment,text
0,joy,on days when i feel close to my partner and ot...
1,fear,every time i imagine that someone i love or i ...
2,anger,when i had been obviously unjustly treated and...
3,sadness,when i think about the short time that we live...
4,disgust,at a gathering i found myself involuntarily si...


In [ ]:
df1['sentiment'].value_counts()

joy        1088
fear       1088
anger      1085
sadness    1081
disgust    1079
guilt      1074
shame      1070
Name: sentiment, dtype: int64

In [ ]:
df2 = df1
df2['sign'] = np.where(df1['sentiment'] == 'joy' , 1 , 0)
df2.head()

,sentiment,text,sign
0,joy,on days when i feel close to my partner and ot...,1
1,fear,every time i imagine that someone i love or i ...,0
2,anger,when i had been obviously unjustly treated and...,0
3,sadness,when i think about the short time that we live...,0
4,disgust,at a gathering i found myself involuntarily si...,0


In [ ]:
df2['sign'].value_counts()

0    6477
1    1088
Name: sign, dtype: int64

In [ ]:
# TEXT PREPROCESSING
#lower case
df2['text'] = df2['text'].apply(lambda x : " ".join([x.lower() for x in x.split()]))

In [ ]:
#replace special characters
df2['text']=df2['text'].str.replace('[^\w\s]','')
df2.head()

,sentiment,text,sign
0,joy,on days when i feel close to my partner and ot...,1
1,fear,every time i imagine that someone i love or i ...,0
2,anger,when i had been obviously unjustly treated and...,0
3,sadness,when i think about the short time that we live...,0
4,disgust,at a gathering i found myself involuntarily si...,0


In [ ]:
#remove stop words
import nltk
nltk.download('stopwords')
stop = stopwords.words('english')
df2['text'] = df2['text'].apply(lambda x : " ".join(x for x in x.split() if x not in stop))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#stemming
st = PorterStemmer()
df2['text'] = df2['text'].apply(lambda x : " ".join(st.stem(x) for x in x.split()))

In [ ]:
df2['text'].head()

0    day feel close partner friend feel peac also e...
1    everi time imagin someon love could contact se...
2                obvious unjustli treat possibl elucid
3    think short time live relat period life think ...
4    gather found involuntarili sit next two peopl ...
Name: text, dtype: object

In [ ]:
#lemmatize
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

df2['text'] = df2['text'].apply(lambda x: " ".join([lemmatizer.lemmatize(x) for word in x.split()]))
df2.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,sentiment,text,sign
0,joy,day feel close partner friend feel peac also e...,1
1,fear,everi time imagin someon love could contact se...,0
2,anger,obvious unjustli treat possibl elucid obvious ...,0
3,sadness,think short time live relat period life think ...,0
4,disgust,gather found involuntarili sit next two peopl ...,0


In [ ]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest= train_test_split(df2['text'], df2['sign'], test_size=0.3, stratify = df2['sign'] ,  random_state=100)

print(xtrain.shape)
print(xtest.shape)
print(ytrain.shape)
print(ytest.shape)

(5295,)
(2270,)
(5295,)
(2270,)


In [ ]:
df2['sign'].value_counts()

0    6477
1    1088
Name: sign, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
ytrain=lb.fit_transform(ytrain)
ytest=lb.transform(ytest)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

tfvect=TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfvect=tfvect.fit(df2['text'])

xtrain_new=tfvect.transform(xtrain)
xtest_new=tfvect.transform(xtest)

In [ ]:
xtest.head()

3875    live room lone quiet neigbourhood night silent...
5305    childhood lie sister found childhood lie siste...
7562    home heard loud sound spit outsid door thought...
7115    abl help friend studi english abl help friend ...
888     got fight chap front famili hous valu properti...
Name: text, dtype: object

In [ ]:
def train_model(classifier, xtrain, ytrain, xtest, ytest):
    mod=classifier.fit(xtrain, ytrain)
    predictions=mod.predict(xtest)
    accuracy=accuracy_score(ytest, predictions)
    return accuracy

In [ ]:
from sklearn import naive_bayes
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

accuracy1 = train_model(naive_bayes.MultinomialNB(), xtrain_new, ytrain, xtest_new, ytest)

print(accuracy1)

0.8744493392070485


In [ ]:


accuracy = train_model(LogisticRegression(), xtrain_new, ytrain, xtest_new, ytest)

print(accuracy)

0.9


In [ ]:
from sklearn.ensemble import RandomForestClassifier

accuracy = train_model(RandomForestClassifier(max_depth = 10 , random_state = 0) , xtrain_new , ytrain , xtest_new , ytest)

print(accuracy)

0.8607929515418502


In [ ]:
# clf -> classifier
clf = LogisticRegression(class_weight = {1:6 , 0:1}).fit(xtrain_new, ytrain)
predictions = clf.predict(xtest_new)
accuracy = accuracy_score(ytest, predictions)
print(accuracy)

0.8903083700440528


In [ ]:
rf = RandomForestClassifier(class_weight= {0:1 , 1:6}).fit(xtrain_new, ytrain)
predictions = rf.predict(xtest_new)
accuracy = accuracy_score(ytest, predictions)
print(accuracy)

0.9149779735682819


In [ ]:
def text_cleaner(sentence):
    import re
    from nltk.tokenize import word_tokenize
    import nltk
    nltk.download('punkt')

    s = {'v2' : [str(sentence)]}
    ef = pd.DataFrame(data = s)
    ef['v2'] = ef['v2'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    ef['v2'] = ef['v2'].str.replace('[^\w\s]','')
    ef['v2'] = ef['v2'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    ef['v2'] = ef['v2'].apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in x.split()]))
    ef['v2'] = ef.apply(lambda x: word_tokenize(x['v2']), axis=1)
    ef['v2'] = [" ".join(x) for x in ef['v2'].values]
    
    #from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
    #tfvect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
    x = tfvect.transform(ef['v2'])
   # x = t.fit_transform(sentence)
    return x

In [ ]:
s = "hate"
s = text_cleaner(s)
print(s)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
  (0, 2089)	1.0


In [ ]:
sentence = "good"
prediction = rf.predict(text_cleaner(sentence))
prediction1 = rf.predict_proba(text_cleaner(sentence))
print(prediction)
print(prediction1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[0]
[[0.76 0.24]]


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(ytest , predictions))

              precision    recall  f1-score   support

           0       0.92      0.99      0.95      1944
           1       0.88      0.47      0.61       326

    accuracy                           0.91      2270
   macro avg       0.90      0.73      0.78      2270
weighted avg       0.91      0.91      0.90      2270

